# Dataobjects by example

## Basic

The simplest possible usage is:

In [1]:
from recordclass import dataobject, datatype

class Empty(dataobject):
     pass

In [2]:
Empty()

Empty()

In [3]:
Empty() == Empty()

True

In [4]:
Empty() is Empty()

False

Consider simple 2d data point:

In [5]:
class Point(dataobject):
    x:int
    y:int

In [6]:
p = Point(1,2)
p

Point(x=1, y=2)

Keyword arguments are allowed too:

In [11]:
p = Point(x=1,y=2)
p

Point(x=1, y=2)

For performance reason you can disable them:

In [14]:
class PointKW(dataobject):
    x:int
    y:int
    __options__ = {'argsonly':True}
    
try:
    PointKW(x=1,y=2)
except TypeError as err:
    print(err)

invalid kwargs


You can define default values. In this case keyword arguments are allowed:

In [15]:
class PointDefaults(dataobject):
    x:int = 0
    y:int = 0
    
p = PointDefaults(x=1,y=2)
p

PointDefaults(x=1, y=2)

If you specify default values, then you are allowed to write:

In [18]:
p = PointDefaults()
print(p)
p = PointDefaults(1)
print(p)

PointDefaults(x=0, y=0)
PointDefaults(x=1, y=0)


In [12]:
from recordclass import datatuple
from recordclass.utils import number_of_dataslots

class A(datatuple):
    x:int
    y:int

print(number_of_dataslots(A))        

2


In [15]:
a=A(1,2,3,4,5)
print(len(a))
a

5


A(x=1, y=2, 3, 4, 5)

In [16]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__annotations__': {'x': int, 'y': int},
              '__new__': <staticmethod at 0x7f10e853f320>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__fields__': ('x', 'y'),
              'x': <recordclass.dataobject.itemgetset at 0x7f10e85d6a70>,
              'y': <recordclass.dataobject.itemgetset at 0x7f10e85d6790>})